<a href="https://colab.research.google.com/github/cchummer/ml-dl-scratch/blob/main/NN_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Simple Models From Scratch
Inspired by lesson 5 of the fastai course. Using the kaggle titanic dataset to create first a simple linear model and add layers to create a neural network.  Data prep is shared/done first in the sections below.

# New Section

In [ ]:
!pip install torch numpy pandas sympy fastai

In [ ]:
import torch, numpy as np, pandas as pd, sympy

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML+DL/kaggle_titanic_train.csv')
df

NameError: name 'pd' is not defined

In [ ]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
modes = df.mode().iloc[0] # Break ties
modes

PassengerId                      1
Survived                       0.0
Pclass                         3.0
Name           Abbing, Mr. Anthony
Sex                           male
Age                           24.0
SibSp                          0.0
Parch                          0.0
Ticket                        1601
Fare                          8.05
Cabin                      B96 B98
Embarked                         S
Name: 0, dtype: object

In [ ]:
df.fillna(modes, inplace=True)
df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [ ]:
df.describe() # seems same results as df.describe(include=(np.number))

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,LogFare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,28.566970,0.523008,0.381594,32.204208,2.962246
std,257.353842,0.486592,0.836071,13.199572,1.102743,0.806057,49.693429,0.969048
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,2.187218
50%,446.000000,0.000000,3.000000,24.000000,0.000000,0.000000,14.454200,2.737881
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000,3.465736
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,6.240917


In [ ]:
# Address large variance + one-tailed'ness of Fare:
df['LogFare'] = np.log(df['Fare']+1)

In [ ]:
df.describe(include=[object]) # for non-numeric columns

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,891,891
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,691,646


In [ ]:
og_cols = df.columns.tolist()

df = pd.get_dummies(df, columns=["Sex", "Pclass", "Embarked"])

new_columns = [col for col in df.columns if col not in og_cols]

df.columns

Index(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'LogFare', 'Sex_female', 'Sex_male', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [ ]:
new_columns

['Sex_female',
 'Sex_male',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

In [ ]:
df.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,LogFare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,B96 B98,2.110213,False,True,False,False,True,False,False,True
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,4.280593,True,False,True,False,False,True,False,False
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,2.188856,True,False,False,False,True,False,False,True
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,3.990834,True,False,True,False,False,False,False,True
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,B96 B98,2.202765,False,True,False,False,True,False,False,True


In [ ]:
# Create independent and dependent variables
from torch import tensor

tens_dep = tensor(df.Survived)
tens_dep

tensor([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
        0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
        1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,

In [ ]:
df.dtypes

PassengerId      int64
Survived         int64
Name            object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
LogFare        float64
Sex_female        bool
Sex_male          bool
Pclass_1          bool
Pclass_2          bool
Pclass_3          bool
Embarked_C        bool
Embarked_Q        bool
Embarked_S        bool
dtype: object

In [ ]:
indep_cols = ['Age', 'SibSp', 'Parch', 'LogFare'] + new_columns # numeric columns + dummies we created. TODO: fully automate to grab numeric columns from df

indep_cols

['Age',
 'SibSp',
 'Parch',
 'LogFare',
 'Sex_female',
 'Sex_male',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

In [ ]:
df[indep_cols].dtypes

Age           float64
SibSp           int64
Parch           int64
LogFare       float64
Sex_female       bool
Sex_male         bool
Pclass_1         bool
Pclass_2         bool
Pclass_3         bool
Embarked_C       bool
Embarked_Q       bool
Embarked_S       bool
dtype: object

In [ ]:
df[indep_cols] = df[indep_cols].astype(float) # For some reason could not get tensor(df[indep_cols].values) to work even though theyre all numeric columns... this fixed it
df[indep_cols].dtypes

Age           float64
SibSp         float64
Parch         float64
LogFare       float64
Sex_female    float64
Sex_male      float64
Pclass_1      float64
Pclass_2      float64
Pclass_3      float64
Embarked_C    float64
Embarked_Q    float64
Embarked_S    float64
dtype: object

In [ ]:
indep_array = df[indep_cols].values
print("Data type of indep_array:", indep_array.dtype)

Data type of indep_array: float64


In [ ]:
tens_indep = tensor(indep_array, dtype=torch.float)

tens_indep

tensor([[22.,  1.,  0.,  ...,  0.,  0.,  1.],
        [38.,  1.,  0.,  ...,  1.,  0.,  0.],
        [26.,  0.,  0.,  ...,  0.,  0.,  1.],
        ...,
        [24.,  1.,  2.,  ...,  0.,  0.,  1.],
        [26.,  0.,  0.,  ...,  1.,  0.,  0.],
        [32.,  0.,  0.,  ...,  0.,  1.,  0.]])

In [ ]:
tens_indep.shape

torch.Size([891, 12])

# New Section

In [ ]:
torch.manual_seed(442)

# Start off with random weights
n_coeff = tens_indep.shape[1]
coeffs = torch.rand(n_coeff)-0.5
coeffs

tensor([-0.4629,  0.1386,  0.2409, -0.2262, -0.2632, -0.3147,  0.4876,  0.3136,
         0.2799, -0.4392,  0.2103,  0.3625])

In [ ]:
tens_indep*coeffs

tensor([[-0.1273,  0.0173,  0.0000,  ..., -0.0000,  0.0000,  0.3625],
        [-0.2199,  0.0173,  0.0000,  ..., -0.4392,  0.0000,  0.0000],
        [-0.1504,  0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.3625],
        ...,
        [-0.1389,  0.0173,  0.0803,  ..., -0.0000,  0.0000,  0.3625],
        [-0.1504,  0.0000,  0.0000,  ..., -0.4392,  0.0000,  0.0000],
        [-0.1852,  0.0000,  0.0000,  ..., -0.0000,  0.2103,  0.0000]])

In [ ]:
# Age column is way out of scale. Make every column a ratio
vals,indices = tens_indep.max(dim=0)
tens_indep = tens_indep / vals

In [ ]:
from fastai.data.transforms import RandomSplitter

trn_split, val_split = RandomSplitter(seed=42)(df)

In [ ]:
trn_indep,val_indep = tens_indep[trn_split], tens_indep[val_split]
trn_dep,val_dep = tens_dep[trn_split], tens_dep[val_split]

len(trn_indep), len(val_indep)

(713, 178)

# Linear Model

In [ ]:
def lin_calc_preds(coeffs, indeps): return torch.sigmoid((indeps*coeffs).sum(axis=1)) # Wrap in sigmoid to squish final outputs between 0 and 1, since this is the only layer
def lin_calc_loss(coeffs, indeps, deps): return torch.abs(lin_calc_preds(coeffs, indeps)-deps).mean()

In [ ]:
coeffs.requires_grad_()

tensor([-0.4629,  0.1386,  0.2409, -0.2262, -0.2632, -0.3147,  0.4876,  0.3136,
         0.2799, -0.4392,  0.2103,  0.3625], requires_grad=True)

In [ ]:
def lin_update_coeffs(coeffs, lr):
    coeffs.sub_(coeffs.grad * lr)
    coeffs.grad.zero_()

In [ ]:
def lin_one_epoch(coeffs, lr):
    loss = lin_calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): lin_update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")

In [ ]:
def lin_init_coeffs(): return (torch.rand(n_coeff)-0.5).requires_grad_()

In [ ]:
def lin_train_model(epochs=30, lr=0.01):
    torch.manual_seed(442)
    coeffs = lin_init_coeffs()
    for i in range(epochs): lin_one_epoch(coeffs, lr=lr)
    return coeffs

In [ ]:
coeffs = lin_train_model(lr=100)

0.503; 0.326; 0.294; 0.207; 0.201; 0.199; 0.198; 0.197; 0.196; 0.196; 0.196; 0.195; 0.195; 0.195; 0.195; 0.195; 0.195; 0.195; 0.194; 0.194; 0.194; 0.194; 0.194; 0.194; 0.194; 0.194; 0.194; 0.194; 0.194; 0.194; 

In [ ]:
def lin_show_coeffs(): return dict(zip(indep_cols, coeffs.requires_grad_(False)))

In [ ]:
# For each pred, if was >.5, check if corresponding dependent value was true/1, get ratio of matches across validation set
def lin_acc(coeffs): return (val_dep.bool()==(lin_calc_preds(coeffs, val_indep)>0.5)).float().mean()

lin_acc(coeffs)

tensor(0.8258)

In [ ]:
lin_show_coeffs()

{'Age': tensor(-1.5097),
 'SibSp': tensor(-1.1564),
 'Parch': tensor(-0.4253),
 'LogFare': tensor(0.2456),
 'Sex_female': tensor(8.4482),
 'Sex_male': tensor(-10.3873),
 'Pclass_1': tensor(3.8377),
 'Pclass_2': tensor(2.1304),
 'Pclass_3': tensor(-6.2481),
 'Embarked_C': tensor(1.4744),
 'Embarked_Q': tensor(2.1067),
 'Embarked_S': tensor(-4.8086)}

# Neural Network - Adding a Layer

In [ ]:
trn_dep = trn_dep[:,None]
val_dep = val_dep[:,None]

In [ ]:
import torch.nn.functional as F

In [ ]:
def nn_calc_preds(coeffs, indeps):
  l1, l2, const = coeffs
  res = F.relu(indeps@l1)
  res = res@l2 + const
  return torch.sigmoid(res)

In [ ]:
def nn_calc_loss(coeffs, indeps, deps): return torch.abs(nn_calc_preds(coeffs, indeps)-deps).mean()

In [ ]:
def nn_init_coeffs(n_hidden=20):
    layer1 = (torch.rand(n_coeff, n_hidden)-0.5)/n_hidden
    layer2 = torch.rand(n_hidden, 1)-0.3

    const = torch.rand(1)[0]

    return layer1.requires_grad_(),layer2.requires_grad_(),const.requires_grad_()

In [ ]:
def nn_update_coeffs(coeffs, lr):
  for layer in coeffs:
    layer.sub_(layer.grad * lr)
    layer.grad.zero_()

In [ ]:
def nn_one_epoch(coeffs, lr):
    loss = nn_calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): nn_update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")

In [ ]:
def nn_train_model(epochs=30, lr=0.01):
    torch.manual_seed(442)
    coeffs = nn_init_coeffs()
    for i in range(epochs): nn_one_epoch(coeffs, lr=lr)
    return coeffs

In [ ]:
coeffs = nn_train_model(lr=1.4)

0.544; 0.533; 0.521; 0.503; 0.483; 0.459; 0.429; 0.394; 0.358; 0.327; 0.302; 0.284; 0.271; 0.261; 0.253; 0.246; 0.242; 0.238; 0.234; 0.231; 0.229; 0.227; 0.226; 0.224; 0.223; 0.222; 0.220; 0.219; 0.218; 0.218; 

In [ ]:
def nn_acc(coeffs): return (val_dep.bool()==(nn_calc_preds(coeffs, val_indep)>0.5)).float().mean()

nn_acc(coeffs)

tensor(0.7921)

# Deep Learning - More Layers

In [ ]:
def dl_init_coeffs():
    hiddens = [10, 10]  # hidden layer sizes
    sizes = [n_coeff] + hiddens + [1]

    n = len(sizes)

    layers = [(torch.rand(sizes[i], sizes[i+1])-0.3)/sizes[i+1]*4 for i in range(n-1)]
    #layers = [(torch.rand(sizes[i], sizes[i+1])-0.3) for i in range(n-1)]

    for layer in layers:
        array = layer.detach().numpy()

    consts = [(torch.rand(1)[0]-0.5)*0.1 for i in range(n-1)]

    for l in layers+consts: l.requires_grad_()
    return layers,consts

In [ ]:
def dl_calc_preds(coeffs, indeps):
    layers,consts = coeffs
    n = len(layers)
    res = indeps
    for i,l in enumerate(layers):
        res = res@l + consts[i]
        if i!=n-1: res = F.relu(res)
    return torch.sigmoid(res)

In [ ]:
def dl_calc_loss(coeffs, indeps, deps): return torch.abs(dl_calc_preds(coeffs, indeps)-deps).mean()

In [ ]:
def dl_update_coeffs(coeffs, lr):
    layers,consts = coeffs
    for layer in layers+consts:
        layer.sub_(layer.grad * lr)
        layer.grad.zero_()

In [ ]:
def dl_one_epoch(coeffs, lr):
    loss = dl_calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): dl_update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")

In [ ]:
def dl_train_model(epochs=30, lr=0.01):
    torch.manual_seed(442)
    coeffs = dl_init_coeffs()
    for i in range(epochs): dl_one_epoch(coeffs, lr=lr)
    return coeffs

In [ ]:
coeffs = dl_train_model(lr=4)

torch.Size([12, 10])
[[-0.10516002  0.13543165  0.17636032 -0.01046164 -0.02529917 -0.04588623
   0.2750438   0.2054275   0.1919533  -0.09566371]
 [ 0.16411555  0.22500536  0.14886792  0.17296953 -0.0630006   0.07961871
   0.00667722 -0.01643815  0.09956107  0.1146451 ]
 [ 0.05865591  0.20528686  0.02050271 -0.02203057  0.2131395   0.13169079
   0.17444453 -0.02489584  0.24204583 -0.02696657]
 [-0.01246278  0.12587368  0.24289398  0.19335492  0.1133882   0.05984907
   0.2541579  -0.0563098  -0.10207782  0.13788864]
 [-0.1059361  -0.04157085  0.02811494  0.0460134   0.26728037  0.2389206
  -0.11396539  0.16391027 -0.04072924  0.24738899]
 [ 0.12356482  0.11162112  0.01168277  0.0047092   0.27445742 -0.02134726
  -0.02008889  0.04918652  0.21345624 -0.11901353]
 [-0.02244551  0.1156882  -0.10681019  0.22900131  0.03618519  0.21772905
  -0.05790269  0.12449102 -0.0346312   0.08978853]
 [ 0.01440942  0.04619877  0.02729776  0.2274561   0.22973366  0.27088895
   0.06685016 -0.04305465  0.13

In [ ]:
def dl_acc(coeffs): return (val_dep.bool()==(dl_calc_preds(coeffs, val_indep)>0.5)).float().mean()

dl_acc(coeffs)

tensor(0.8258)